In [1]:
#%matplotlib inline
import numpy as np

from fatiando import gravmag, mesher, utils, gridder

#from fatiando.gravmag import prism
from fatiando.gravmag import _prism_numpy as prism

from fatiando.mesher import Prism, Polygon
from fatiando.utils import ang2vec, si2nt
from fatiando.gridder import regular
from fatiando.vis import mpl

In [2]:
def Euler2D(tf, Dytf, Dztf, y, z, si, npontos):
    '''
    Deconvolução de Euler 2D
    
    sistema de coordenadas
      x: dimensão infinita
      y: direção ao longo do perfil
      z: aponta para baixo
    
    input
      tf: array com a anomalia de campo total (ACT)
      Dytf: array com a derivada espacial da ACT na direção y
      Dztf: array com a derivada espacial da ACT na direção z
      y: array com a coordenadas das observações ao longo do perfil
      si: índice estrutural
      npontos: número de pontos em cada janela
      
    output
      y0: array com as coordenadas y estimadas para cada janela
          de dados
      z0: array com as coordenadas z estimadas para cada janela
          de dados
      b: array com os níveis de base estimados para cada janela
          de dados

    '''

    y0 = np.zeros(len(y) + 1 - npontos)
    z0 = np.zeros(len(y) + 1 - npontos)
    b = np.zeros(len(y) + 1 - npontos)
    
    vsi = si*np.ones(npontos)

    # Janela móvel
    for i in range(len(y) + 1 - npontos):
        S = np.vstack((Dytf[i:i+npontos],Dztf[i:i+npontos],vsi)).T
        s = Dytf[i:i+npontos]*y[i:i+npontos] + \
            Dztf[i:i+npontos]*z[i:i+npontos] + \
            vsi*tf[i:i+npontos]
        STS = S.T.dot(S)
        STs = S.T.dot(s)
        # Coordenadas estimadas
        p = np.linalg.solve(STS,STs)
        y0[i] = p[0]
        z0[i] = p[1]
        b[i]  = p[2]

    return y0, z0, b

In [3]:
''' Coordenadas das observações e do modelo'''

ymin = 0.0
ymax = 100000.0
ny = 201
dy = (ymax - ymin)/(ny - 1)
L = ymax - ymin
y = np.linspace(ymin, ymax, ny)
x = np.zeros_like(y)
z = np.zeros_like(y)-150.0

zmin = -2000.0
zmax = 8000.0
xmin = -100000.0
xmax =  100000.0

area = [ymin, ymax, zmax, zmin]

incs, decs = 18.0, 10.0 # inclinação e declinação do dique
inc, dec = -39.8, -22.5 # inclinação e declinação do campo geomagnético

In [4]:
''' Dique '''

# Topo do dique
d = np.array([[63000.0, 700.0]])

dique = []
Ldique = 500.0
poligono_dique = []
for i, p in enumerate(d):
    dique.append(mesher.Prism(xmin, xmax, 
                              p[0] - Ldique, p[0] + Ldique,
                              p[1], 10000, {'magnetization': utils.ang2vec(3.0, incs, decs)}))
    poligono_dique.append(mesher.Polygon([[p[0] - Ldique, p[1]],
                                            [p[0] + Ldique, p[1]],
                                            [p[0] + Ldique, 100000],
                                            [p[0] - Ldique, 100000]]))
    

# Visualização do dique
lambida = 0.10
mpl.figure()
mpl.title('Dique')
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
for i, p in enumerate(poligono_dique):
    mpl.polygon(p, style='-k', linewidth=1)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)')
mpl.ylabel('z (km)')
mpl.m2km()
mpl.show()

In [5]:
''' Anomalia de campo total (ACT) '''

tf = prism.tf(x, y, z, dique, inc, dec) + 30.0

In [6]:
''' Visualização da anomalia de campo total '''

mpl.close('all')
mpl.figure()

lambida = 0.10

mpl.subplot(2,1,1)
mpl.plot(0.001*y, tf, 'r-')
mpl.plot(0.001*y, tf, 'ro', markersize=3)
vmax = np.max(tf)
vmin = np.min(tf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('ACT (nT)')

mpl.subplot(2,1,2)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
for i, p in enumerate(poligono_dique):
    mpl.polygon(p, style='-k', linewidth=1)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)')
mpl.ylabel('z (km)')
mpl.m2km()

mpl.show()

In [7]:
''' Derivadas espaciais da ACT '''

delta = 10.0

# Derivadas numéricas
Dytf  = prism.tf(x, y + delta, z, dique, inc, dec)
Dytf -= prism.tf(x, y - delta, z, dique, inc, dec)
Dytf  = Dytf/(2.0*delta) # nT/km

Dztf  = prism.tf(x, y, z + delta, dique, inc, dec)
Dztf -= prism.tf(x, y, z - delta, dique, inc, dec)
Dztf  = Dztf/(2.0*delta) # nT/km

# Visualização da anomalia de campo total + derivadas em y e z
mpl.close('all')
mpl.figure()

lambida = 0.10

mpl.subplot(3,1,1)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.plot(0.001*y, tf, 'k-')
mpl.plot(0.001*y, tf, 'ko', markersize=3)
vmax = np.max(tf)
vmin = np.min(tf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('ACT (nT)')

mpl.subplot(3,1,2)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.plot(0.001*y, 1000.0*Dytf, 'k-')
mpl.plot(0.001*y, 1000.0*Dytf, 'ko', markersize=3)
vmax = np.max(1000.0*Dytf)
vmin = np.min(1000.0*Dytf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('dACT/dy (nT/km)')

mpl.subplot(3,1,3)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.plot(0.001*y, 1000.0*Dztf, 'k-')
mpl.plot(0.001*y, 1000.0*Dztf, 'ko', markersize=3)
vmax = np.max(1000.0*Dztf)
vmin = np.min(1000.0*Dztf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin,0.001* ymax)
mpl.ylabel('dACT/dz (nT/km)')
mpl.xlabel('y (km)')

mpl.show()

In [31]:
''' Parâmetros da Deconvolução de Euler '''

npontos = 75 # número de pontos por janela
#npontos = ny # apenas uma janela

# coordenadas do centro de cada janela
dymedio = 0.5*(y[npontos-1] - y[0])
ymedio = np.linspace(ymin + dymedio, 
                     ymax - dymedio, 
                     len(y) + 1 - npontos)

n = [0.5,1,1.5,2,3] # lista de índices estruturais
colors = ['ro','go','bo','yo','ko']
labels = ['0.5','1','1.5','2','3']

mpl.close('all')
mpl.figure()

lambida = 0.10

mpl.subplot(2,1,1)
mpl.plot(0.001*y, tf, 'r-')
mpl.plot(0.001*y, tf, 'ro', markersize=3)
mpl.plot(0.001*y[:npontos], tf[:npontos], 'b-', markersize=5)
mpl.plot(0.001*y[:npontos], tf[:npontos], 'bo', markersize=5)
vmax = np.max(tf)
vmin = np.min(tf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('ACT (nT)')

mpl.subplot(2,1,2)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
for i, p in enumerate(poligono_dique):
    mpl.polygon(p, style='-k', linewidth=1)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)')
mpl.ylabel('z (km)')
mpl.m2km()

mpl.show()

In [32]:
''' Deconvolução de euler '''

estimativas_y0 = []
estimativas_z0 = []
estimativas_b  = []

for j in range(len(n)):
    
    y0, z0, b = Euler2D(tf, Dytf, Dztf, y, z, n[j], npontos)
    estimativas_y0.append(y0)
    estimativas_z0.append(z0)
    estimativas_b.append(b)

In [19]:
''' Visualização das estimativas + ACT '''

lambida = 0.10

# Todas juntas
mpl.close('all')
mpl.figure()

mpl.subplot(2,1,1)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.plot(0.001*y, tf, 'r-')
mpl.plot(0.001*y, tf, 'ro', markersize=3)
mpl.plot(0.001*y[:npontos], tf[:npontos], 'b-')
mpl.plot(0.001*y[:npontos], tf[:npontos], 'bo', markersize=5)
vmax = np.max(tf)
vmin = np.min(tf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('ACT (nT)')

mpl.subplot(2,1,2)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
for i, p in enumerate(poligono_dique):
    mpl.polygon(p, style='-k', linewidth=1)
for j in range(len(n)):
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)')
mpl.ylabel('z (km)')
mpl.m2km()

mpl.show()

# Separadas
for j in range(len(n)):
    mpl.close('all')
    mpl.figure()

    mpl.subplot(2,1,1)
    mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
    mpl.plot(0.001*y, tf, 'r-')
    mpl.plot(0.001*y, tf, 'ro', markersize=3)
    mpl.plot(0.001*y[:npontos], tf[:npontos], 'b-')
    mpl.plot(0.001*y[:npontos], tf[:npontos], 'bo', markersize=5)
    vmax = np.max(tf)
    vmin = np.min(tf)
    dv = lambida*(vmax - vmin)
    mpl.ylim(vmin - dv, vmax + dv)
    mpl.xlim(0.001*ymin, 0.001*ymax)
    mpl.ylabel('ACT (nT)')

    mpl.subplot(2,1,2)
    mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
    for i, p in enumerate(poligono_dique):
        mpl.polygon(p, style='-k', linewidth=1)
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
    dv = lambida*(zmax - zmin)
    mpl.ylim(zmax, zmin - dv)
    mpl.xlim(ymin, ymax)
    mpl.xlabel('y (km)')
    mpl.ylabel('z (km)')
    mpl.m2km()

    mpl.show()

In [33]:
''' Visualização das estimativas y em patamares '''

lambida = 0.10

# Todas juntas
mpl.close('all')
mpl.figure()

mpl.subplot(2,1,1)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.paths([[0.001*ymin, 0.001*d[0,0]]], [[0.001*ymax, 0.001*d[0,0]]], style='-k', linewidth=1.0)

for j in range(len(n)):
    mpl.plot(0.001*ymedio, 0.001*estimativas_y0[j], colors[j], label=labels[j], markersize=3)

vmax = 0.001*np.max(estimativas_y0)
vmin = 0.001*np.min(estimativas_y0)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)

mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('$y_{0}$ (km)')

mpl.subplot(2,1,2)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
for i, p in enumerate(poligono_dique):
    mpl.polygon(p, style='-k', linewidth=1)
for j in range(len(n)):
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)')
mpl.ylabel('z (km)')
mpl.m2km()

mpl.show()

# Separadas
for j in range(len(n)):
    mpl.close('all')
    mpl.figure()

    mpl.subplot(2,1,1)
    mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
    mpl.paths([[0.001*ymin, 0.001*d[0,0]]], [[0.001*ymax, 0.001*d[0,0]]], style='-k', linewidth=1.0)
    mpl.plot(0.001*ymedio, 0.001*estimativas_y0[j], colors[j], label=labels[j], markersize=3)

    vmax = 0.001*np.max(estimativas_y0)
    vmin = 0.001*np.min(estimativas_y0)
    dv = lambida*(vmax - vmin)
    mpl.ylim(vmin - dv, vmax + dv)
    
    mpl.xlim(0.001*ymin, 0.001*ymax)
    mpl.ylabel('$y_{0}$ (km)')

    mpl.subplot(2,1,2)
    mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
    for i, p in enumerate(poligono_dique):
        mpl.polygon(p, style='-k', linewidth=1)
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
    dv = lambida*(zmax - zmin)
    mpl.ylim(zmax, zmin - dv)
    mpl.xlim(ymin, ymax)
    mpl.xlabel('y (km)')
    mpl.ylabel('z (km)')
    mpl.m2km()

    mpl.show()

In [23]:
''' Visualização das estimativas z em patamares '''

lambida = 0.10

# Todas juntas
mpl.close('all')
mpl.figure()

mpl.subplot(2,1,1)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
for j in range(len(n)):
    mpl.plot(0.001*ymedio, 0.001*estimativas_z0[j], colors[j], label=labels[j], markersize=3)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('$z_{0}$ (km)')

mpl.subplot(2,1,2)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
for i, p in enumerate(poligono_dique):
    mpl.polygon(p, style='-k', linewidth=1)
for j in range(len(n)):
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)')
mpl.ylabel('z (km)')
mpl.m2km()

mpl.show()

# Separadas
for j in range(len(n)):
    mpl.close('all')
    mpl.figure()

    mpl.subplot(2,1,1)
    mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
    mpl.plot(0.001*ymedio, 0.001*estimativas_z0[j], colors[j], label=labels[j], markersize=3)
    mpl.xlim(0.001*ymin, 0.001*ymax)
    mpl.ylabel('$z_{0}$ (km)')

    mpl.subplot(2,1,2)
    mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
    for i, p in enumerate(poligono_dique):
        mpl.polygon(p, style='-k', linewidth=1)
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
    dv = lambida*(zmax - zmin)
    mpl.ylim(zmax, zmin - dv)
    mpl.xlim(ymin, ymax)
    mpl.xlabel('y (km)')
    mpl.ylabel('z (km)')
    mpl.m2km()

    mpl.show()

In [13]:
''' Visualização das estimativas '''

lambida = 0.10

# Separadas
for j in range(len(n)):

    mpl.close('all')
    mpl.figure()

    mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
    for i, p in enumerate(poligono_dique):
        mpl.polygon(p, style='-k', linewidth=1)
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
    dv = lambida*(zmax - zmin)
    mpl.ylim(zmax, zmin - dv)
    mpl.xlim(ymin, ymax)
    mpl.xlabel('y (km)')
    mpl.ylabel('z (km)')
    mpl.m2km()
    mpl.show()

# Todas juntas
mpl.close('all')
mpl.figure()

mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
for i, p in enumerate(poligono_dique):
    mpl.polygon(p, style='-k', linewidth=1)
for j in range(len(n)):
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)')
mpl.ylabel('z (km)')
mpl.m2km()
mpl.show()
